<a href="https://colab.research.google.com/github/gabiUfsc/QUANTIcolab/blob/main/analisador_estat_stico_completo_quanti_norm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import os
import pandas as pd
import unicodedata

# =================================================================
# 1. MAPEAMENTO DOS ARQUIVOS (NOMES EXATOS VERIFICADOS)
# =================================================================
GRUPOS = {
    'Extrema Direita': [
        'olavodcarvalho.txt',
        'sergio_moro1_25.txt',
        'RodConstantino1_25.txt',
        'Nikole_1_25.txt',
        'principe_1_25.txt'
    ],
    'Centro': [
        'elio_gaspari.txt',
        'celso_rochaDebarros.txt',
        'Josias de Souza.txt',
        'reinaldo_1_25.txt',
        'joel_pinheiro.txt'
    ],
    'Esquerda': [
        'jesse_souza.txt',
        'attuch_1_25.txt',
        'xicoSa_1_25.txt',
        'luisCPinto.txt',
        'pauloMLeite1_25.txt'
    ]
}

# =================================================================
# 2. OS 30 LEXEMAS (EIXOS SEMÂNTICOS)
# =================================================================
LEXEMAS = {
    'ALIADO/ALIANÇA': r'\b(aliad[oa][s]?|alian[cç][a-z]*|aliar)\b',
    'AMEAÇAR/AMEAÇA': r'\bamea[cç][a-z]*\b',
    'ARMA/ARMAMENTO': r'\barm(as?|inha|amento|amentista)\b',
    'ATACAR/ATAQUE': r'\b(atac[a-z]*|ataqu[ei][a-z]*)\b',
    'AVANÇAR/AVANÇO': r'\bavan[cç][a-z]*\b',
    'BLOQUEAR/BLOQUEIO': r'\bbloque[a-z]*\b',
    'CAMPANHA': r'\bcampanh[a-z]*\b',
    'COMANDAR/COMANDO': r'\bcomand[a-z]*\b',
    'COMBATER/COMBATE': r'\bcombat[ei][a-z]*\b',
    'DEFENDER/DEFESA': r'\b(defend[a-z]*|defes[a-z]*)\b',
    'DERROTAR/DERROTA': r'\bderrot[a-z]*\b',
    'DESTRUIR/DESTRUIÇÃO': r'\bdestr[ui][a-z]*\b',
    'ESTRATÉGIA': r'\bestrat.g[a-z]*\b',
    'GUERRA': r'\bguerr[a-z]*\b',
    'INIMIGO': r'\b(inimig[oa][s]?|inimiz[a-z]*)\b',
    'INVADIR/INVASÃO': r'\b(invad[a-z]*|invas[a-z]*)\b',
    'LUTAR/LUTA': r'\b(lut[as]|lutou|lutaram|lutando)\b',
    'MANOBRAR/MANOBRA': r'\bmanobr[a-z]*\b',
    'MISSÃO': r'\bmiss.o\b|\bmiss.es\b',
    'MOBILIZAR/MOBILIZAÇÃO': r'\bmobiliz[a-z]*\b',
    'MUNIÇÃO': r'\bmuni[cç][a-z]*\b',
    'PROTEGER/PROTEÇÃO': r'\bprote[cg][a-z]*\b',
    'PUNIÇÃO/PUNIR': r'\bpuni[cç][a-z]*\b',
    'RECUAR/RECUO': r'\brecu[a-z]*\b',
    'RESISTIR/RESISTÊNCIA': r'\bresist[a-z]*\b',
    'SALVAR/SALVAÇÃO': r'\bsalv[a-z]*\b',
    'SANÇÃO': r'\bsan[cç][a-z]*\b',
    'TIRO/ATIRAR': r'\b(tiro[s]?|dispar[a-z]*|atir[a-z]*)\b',
    'VENCER/VENCEDOR': r'\bvenc[a-z]*\b',
    'VITÓRIA': r'\bvit.r[a-z]*\b'
}

# =================================================================
# 3. MOTOR DE CÁLCULO
# =================================================================

dados_detalhados = []

print(">>> INICIANDO CÁLCULOS COMPLETOS (QUANTI + NORM)...")

for grupo, arquivos in GRUPOS.items():
    for arquivo in arquivos:
        nome_limpo = arquivo.replace('.txt', '').replace('_', ' ').replace('1 25', '').title()

        if not os.path.exists(arquivo):
            print(f"[ERRO] Arquivo ausente: {arquivo}")
            continue

        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                texto = f.read()
        except:
            with open(arquivo, 'r', encoding='latin-1') as f:
                texto = f.read()

        # 1. Contagem Total de Palavras (N)
        # Removemos pontuação básica para contar palavras reais
        tokens = re.findall(r'\b\w+\b', texto.lower())
        total_palavras_autor = len(tokens)

        if total_palavras_autor == 0:
            print(f"[AVISO] Arquivo vazio: {arquivo}")
            continue

        # 2. Busca de Lexemas
        for categoria, regex in LEXEMAS.items():
            ocorrencias = len(re.findall(regex, texto, re.IGNORECASE))

            if ocorrencias > 0:
                dados_detalhados.append({
                    'Ideologia': grupo,
                    'Autor': nome_limpo,
                    'Lexema': categoria,
                    'Hits_Absolutos': ocorrencias,
                    'Total_Palavras_Autor': total_palavras_autor
                })

# =================================================================
# 4. GERAÇÃO DAS TABELAS ESTATÍSTICAS
# =================================================================

df = pd.DataFrame(dados_detalhados)

if not df.empty:
    # --- CÁLCULO 1: Agrupamento por Ideologia e Lexema ---
    # Somamos os hits de todos os autores do grupo para cada lexema
    df_ideologia = df.groupby(['Ideologia', 'Lexema']).agg({
        'Hits_Absolutos': 'sum',
        'Total_Palavras_Autor': 'sum' # Cuidado: aqui somamos o total de palavras repetido para cada lexema, corrigiremos abaixo
    }).reset_index()

    # Correção do Total de Palavras por Grupo (Para Normalização Correta)
    # Primeiro calculamos o total real de palavras de cada grupo (somando 1x por autor)
    df_unicos = df[['Ideologia', 'Autor', 'Total_Palavras_Autor']].drop_duplicates()
    total_palavras_grupo = df_unicos.groupby('Ideologia')['Total_Palavras_Autor'].sum().to_dict()

    # --- CÁLCULO 2: Métricas Finais (Relativa e Normalizada) ---
    def calcular_metricas(row):
        n_total = total_palavras_grupo[row['Ideologia']]
        hits = row['Hits_Absolutos']

        # Frequência Relativa (%)
        freq_relativa = (hits / n_total) * 100

        # Frequência Normalizada (por 10.000 palavras)
        freq_norm = (hits / n_total) * 10000

        return pd.Series([n_total, freq_relativa, freq_norm])

    df_ideologia[['Total_Palavras_Grupo', 'Freq_Relativa_%', 'Freq_Norm_10k']] = df_ideologia.apply(calcular_metricas, axis=1)

    # Arredondamento para estética
    df_ideologia['Freq_Relativa_%'] = df_ideologia['Freq_Relativa_%'].round(4)
    df_ideologia['Freq_Norm_10k'] = df_ideologia['Freq_Norm_10k'].round(2)

    # --- EXPORTAÇÃO ---

    # 1. Tabela Matriz (Ideologia x Lexema - Normalizada)
    matriz_norm = df_ideologia.pivot(index='Lexema', columns='Ideologia', values='Freq_Norm_10k').fillna(0)
    matriz_norm.to_csv('FINAL_Matriz_Normalizada_10k.csv', encoding='utf-8-sig')

    # 2. Relatório Completo (Lista Longa)
    df_ideologia.to_csv('FINAL_Relatorio_Estatistico_Completo.csv', index=False, encoding='utf-8-sig')

    print("\n" + "="*60)
    print("ESTATÍSTICAS FINAIS GERADAS COM SUCESSO")
    print("="*60)
    print("Total de palavras analisadas por grupo:")
    for grupo, total in total_palavras_grupo.items():
        print(f"  - {grupo}: {total} palavras")

    print("\n>>> ARQUIVOS CRIADOS NA PASTA:")
    print("1. 'FINAL_Matriz_Normalizada_10k.csv'")
    print("   (Esta é a tabela pronta para fazer o GRÁFICO COMPARATIVO)")
    print("2. 'FINAL_Relatorio_Estatistico_Completo.csv'")
    print("   (Contém os cálculos de % e números absolutos para citação)")

else:
    print("Nenhum dado encontrado. Verifique os arquivos.")

>>> INICIANDO CÁLCULOS COMPLETOS (QUANTI + NORM)...

ESTATÍSTICAS FINAIS GERADAS COM SUCESSO
Total de palavras analisadas por grupo:
  - Centro: 70274 palavras
  - Esquerda: 70434 palavras
  - Extrema Direita: 87326 palavras

>>> ARQUIVOS CRIADOS NA PASTA:
1. 'FINAL_Matriz_Normalizada_10k.csv'
   (Esta é a tabela pronta para fazer o GRÁFICO COMPARATIVO)
2. 'FINAL_Relatorio_Estatistico_Completo.csv'
   (Contém os cálculos de % e números absolutos para citação)
